<a href="https://colab.research.google.com/github/horus84/RL-Model-Price-Prediction-/blob/main/RL_FINANCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy pandas yfinance torch sklearn matplotlib

import numpy as np
import pandas as pd
import yfinance as yf
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Configuration
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'TSLA']
START_DATE = '2015-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')
SEQ_LENGTH = 60
PREDICTION_DAYS = 7
BATCH_SIZE = 32
EPOCHS = 100
TRAIN_TEST_SPLIT = 0.8

# LSTM Model Definition
class StockPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(StockPredictor, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Dataset Class
class StockDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence, target = self.sequences[idx]
        return torch.tensor(sequence, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)

def fetch_data(tickers):
    data = yf.download(tickers, start=START_DATE, end=END_DATE, auto_adjust=False)
    adj_close = data.xs('Adj Close', axis=1, level=0)
    return adj_close.dropna()

def preprocess_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    sequences = []
    for i in range(len(scaled_data) - SEQ_LENGTH - PREDICTION_DAYS):
        seq = scaled_data[i:i+SEQ_LENGTH]
        target = scaled_data[i+SEQ_LENGTH:i+SEQ_LENGTH+PREDICTION_DAYS].flatten()
        sequences.append((seq, target))

    return sequences, scaler

def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    for epoch in range(EPOCHS):
        for sequences, targets in train_loader:
            sequences, targets = sequences.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {loss.item():.6f}')

def evaluate_model(model, test_loader, scaler, device):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for sequences, targets in test_loader:
            sequences = sequences.to(device)
            outputs = model(sequences).cpu().numpy()
            predictions.extend(outputs)
            actuals.extend(targets.numpy())

    predictions = scaler.inverse_transform(np.array(predictions))
    actuals = scaler.inverse_transform(np.array(actuals))

    mse = mean_squared_error(actuals, predictions)
    mae = mean_absolute_error(actuals, predictions)
    return mse, mae, predictions, actuals

def plot_results(actuals, predictions, ticker):
    plt.figure(figsize=(12, 6))
    idx = TICKERS.index(ticker)
    actual_series = actuals[:, :, idx].flatten()
    pred_series = predictions[:, :, idx].flatten()
    days = np.arange(len(actual_series))
    plt.plot(days, actual_series, label='Actual')
    plt.plot(days, pred_series, label='Predicted')
    plt.title(f'{ticker} Stock Price Prediction')
    plt.xlabel('Days')
    plt.ylabel('Price')
    plt.legend()
    plt.savefig(f'{ticker}_prediction.png')
    plt.close()

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    # Fetch and prepare data
    data = fetch_data(TICKERS)
    sequences, scaler = preprocess_data(data.values)

    # Split data
    split_idx = int(len(sequences) * TRAIN_TEST_SPLIT)
    train_data = StockDataset(sequences[:split_idx])
    test_data = StockDataset(sequences[split_idx:])

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

    # Initialize model
    model = StockPredictor(
        input_size=data.shape[1],
        hidden_size=64,
        num_layers=2,
        output_size=PREDICTION_DAYS*data.shape[1]
    ).to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Train and evaluate
    train_model(model, train_loader, criterion, optimizer, device)
    mse, mae, predictions, actuals = evaluate_model(model, test_loader, scaler, device)

    print(f'\nEvaluation Metrics:')
    print(f'MSE: {mse:.4f}')
    print(f'MAE: {mae:.4f}')

    # Plot results for each ticker
    for i, ticker in enumerate(TICKERS):
        plot_results(actuals[:, i::len(TICKERS)], predictions[:, i::len(TICKERS)], ticker)

if __name__ == '__main__':
    main()


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Using device: cpu


[*********************100%***********************]  4 of 4 completed


Epoch 1/100, Loss: 0.003126
Epoch 2/100, Loss: 0.001159
Epoch 3/100, Loss: 0.001610
Epoch 4/100, Loss: 0.001323
Epoch 5/100, Loss: 0.000566
Epoch 6/100, Loss: 0.001014
Epoch 7/100, Loss: 0.000748
Epoch 8/100, Loss: 0.000740
Epoch 9/100, Loss: 0.000203
Epoch 10/100, Loss: 0.001273
Epoch 11/100, Loss: 0.000856
Epoch 12/100, Loss: 0.001046
Epoch 13/100, Loss: 0.000728
Epoch 14/100, Loss: 0.000751
Epoch 15/100, Loss: 0.000761
Epoch 16/100, Loss: 0.000619
Epoch 17/100, Loss: 0.000456
Epoch 18/100, Loss: 0.000297
Epoch 19/100, Loss: 0.000519
Epoch 20/100, Loss: 0.000417
Epoch 21/100, Loss: 0.000721
Epoch 22/100, Loss: 0.000592
Epoch 23/100, Loss: 0.000760
Epoch 24/100, Loss: 0.000383
Epoch 25/100, Loss: 0.000621
Epoch 26/100, Loss: 0.000511
Epoch 27/100, Loss: 0.000430
Epoch 28/100, Loss: 0.000303
Epoch 29/100, Loss: 0.000340
Epoch 30/100, Loss: 0.000514
Epoch 31/100, Loss: 0.000481
Epoch 32/100, Loss: 0.000700
Epoch 33/100, Loss: 0.000797
Epoch 34/100, Loss: 0.000444
Epoch 35/100, Loss: 0.0

ValueError: operands could not be broadcast together with shapes (502,28) (4,) (502,28) 